In [33]:
# Dependencies
import pandas as pd
import requests
import json
import os

# Google API Key (maybe later)
from config import gkey

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, gkey)

NameError: name 'target_city' is not defined

In [31]:
# Source files
attendance_raw = 'Resources/national_park_attendance_Prev3Y.xlsx'
park_info = 'Resources/national_park_locations.xlsx'

# Read data and store in dataframe
attendance_data = pd.read_excel(attendance_raw)
park_info_data = pd.read_excel(park_info)


In [32]:
# Limit dataframe to California & Florifa
focus_parks = park_info_data.loc[(park_info_data['State'] == 'CA') | (park_info_data['State'] == 'FL')]
focus_parks.head()

,Park,UnitCode,ParkType,Region,State
25,Big Cypress NPRES,BICY,National Preserve,Southeast,FL
30,Biscayne NP,BISC,National Park,Southeast,FL
41,Cabrillo NM,CABR,National Monument,Pacific West,CA
42,Canaveral NS,CANA,National Seashore,Southeast,FL
56,Castillo de San Marcos NM,CASA,National Monument,Southeast,FL


In [34]:
# Combine the data into a single dataframe  
park_data_merge = pd.merge(focus_parks, attendance_data, how="left", on=["Park", "Park"])
park_data_merge.head()

,Park,UnitCode,ParkType,Region,State,Code,Month,Visitors
0,Big Cypress NPRES,BICY,National Preserve,Southeast,FL,BICY,2020-01-01 00:00:00,152542.0
1,Big Cypress NPRES,BICY,National Preserve,Southeast,FL,BICY,2020-02-01 00:00:00,198012.0
2,Big Cypress NPRES,BICY,National Preserve,Southeast,FL,BICY,2020-03-01 00:00:00,131162.0
3,Big Cypress NPRES,BICY,National Preserve,Southeast,FL,BICY,2020-04-01 00:00:00,46748.0
4,Big Cypress NPRES,BICY,National Preserve,Southeast,FL,BICY,2020-05-01 00:00:00,84674.0


In [4]:
focus_parks.head()

,National Park,Coordinates
0,Biscayne,"{'latitude': 25.65, 'longitude': -80.08}"
1,Channel Islands,"{'latitude': 34.01, 'longitude': -119.42}"
2,Death Valley,"{'latitude': 36.24, 'longitude': -116.82}"
3,Dry Tortugas,"{'latitude': 24.63, 'longitude': -82.87}"
4,Everglades,"{'latitude': 25.32, 'longitude': -80.93}"


In [5]:
park_info_data.head()

,ParkName,UnitCode,ParkType,Region,State
0,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY
1,Acadia NP,ACAD,National Park,Northeast,ME
2,Adams NHP,ADAM,National Historical Park,Northeast,MA
3,African Burial Ground NM,AFBG,National Monument,Northeast,NY
4,Agate Fossil Beds NM,AGFO,National Monument,Midwest,NE
